In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [14]:
# Configure gmaps to use your Google API key. You only have to do this once
gmaps.configure(api_key=g_key)

In [2]:
# Store the CSV saved/created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

City_ID               City Country    Lat     Lng  Max Temp  Humidity  \
0        0  Sosnovo-Ozerskoye      RU  52.52  111.54     56.75        91   
1        1              Kapaa      US  22.08 -159.32     82.40        69   
2        2          Mahebourg      MU -20.41   57.70     69.80        77   
3        3          Cape Town      ZA -33.93   18.42     53.01        87   
4        4            Qaanaaq      GL  77.48  -69.36     46.24        56   

   Cloudiness  Wind Speed   Current Description  Rain (inches)  Snow (inches)  
0          71        4.27         broken clouds           0.00              0  
1          40       12.75      scattered clouds           0.00              0  
2          75       20.80           shower rain           0.00              0  
3          75       16.11  heavy intensity rain           2.54              0  
4          76        3.40         broken clouds           0.00              0

In [3]:
city_data_df["Snow (inches)"].dtypes


dtype('int64')

In [15]:
# Get the maximum temperature. Replace negative temps with 0
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))


In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 86


In [18]:
# Ask the customer rain and snow preferences
rain_ask = input("Do you want it to be raining? (yes/no) ")
snow_ask = input("Do you want it to be snowing? (yes/no) ")

Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [19]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df
if rain_ask == 'no' and snow_ask == 'no':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Max Temp"] <= max_temp) & \
                                                  (preferred_cities_df["Max Temp"] >= min_temp) & \
                                                  (preferred_cities_df["Rain (inches)"] == 0) & \
                                                  (preferred_cities_df["Snow (inches)"] == 0)]
elif rain_ask == 'yes' and snow_ask == 'no':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Max Temp"] <= max_temp) & \
                                                  (preferred_cities_df["Max Temp"] >= min_temp) & \
                                                  (preferred_cities_df["Rain (inches)"] > 0) & \
                                                  (preferred_cities_df["Snow (inches)"] == 0)]
elif rain_ask == 'no' and snow_ask == 'yes':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Max Temp"] <= max_temp) & \
                                                  (preferred_cities_df["Max Temp"] >= min_temp) & \
                                                  (preferred_cities_df["Rain (inches)"] == 0) & \
                                                  (preferred_cities_df["Snow (inches)"] > 0)]
elif rain_ask == 'yes' and snow_ask == 'yes':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Max Temp"] <= max_temp) & \
                                                  (preferred_cities_df["Max Temp"] >= min_temp) & \
                                                  (preferred_cities_df["Rain (inches)"] > 0) & \
                                                  (preferred_cities_df["Snow (inches)"] > 0)]
preferred_cities_df.head(10)

City_ID           City Country    Lat     Lng  Max Temp  Humidity  \
13        13         Albany      US  42.60  -73.97     82.99        65   
25        25        Konongo      GH   6.62   -1.22     71.42        95   
42        42        Merauke      ID  -8.47  140.33     77.72        87   
61        61  Lengshuijiang      CN  27.69  111.43     76.41        93   
107      107     Butaritari      KI   3.07  172.79     83.10        75   
109      109         Padang      ID  -0.95  100.35     77.54        81   
110      110         Atuona      PF  -9.80 -139.03     79.57        79   
126      126         Kozhva      RU  65.11   57.05     67.77        91   
158      158       Lorengau      PG  -2.02  147.27     82.35        79   
166      166    Chaykovskiy      RU  56.77   54.11     66.99        92   

     Cloudiness  Wind Speed Current Description  Rain (inches)  Snow (inches)  
13          100        9.40       moderate rain           2.91              0  
25          100        1.79          light rain           0.23              0  
42           46       15.99          light rain           0.25              0  
61          100        2.64          light rain           0.48              0  
107         100       11.39       moderate rain           2.43              0  
109          70        5.30          light rain           0.33              0  
110          96       18.97          light rain           0.12              0  
126         100       10.02          light rain           0.56              0  
158          84       12.10          light rain           0.11              0  
166          98        1.99          light rain           0.13              0

In [20]:
preferred_cities_df.count()

City_ID                46
City                   46
Country                46
Lat                    46
Lng                    46
Max Temp               46
Humidity               46
Cloudiness             46
Wind Speed             46
Current Description    46
Rain (inches)          46
Snow (inches)          46
dtype: int64

In [21]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Rain (inches)          0
Snow (inches)          0
dtype: int64

In [22]:
# Create DataFrame called cities_hotel_df that adds a hotel in the city 
cities_hotel_df = preferred_cities_df.copy()
cities_hotel_df["Hotel Name"] = ""
cities_hotel_df.head(10)

City_ID           City Country    Lat     Lng  Max Temp  Humidity  \
13        13         Albany      US  42.60  -73.97     82.99        65   
25        25        Konongo      GH   6.62   -1.22     71.42        95   
42        42        Merauke      ID  -8.47  140.33     77.72        87   
61        61  Lengshuijiang      CN  27.69  111.43     76.41        93   
107      107     Butaritari      KI   3.07  172.79     83.10        75   
109      109         Padang      ID  -0.95  100.35     77.54        81   
110      110         Atuona      PF  -9.80 -139.03     79.57        79   
126      126         Kozhva      RU  65.11   57.05     67.77        91   
158      158       Lorengau      PG  -2.02  147.27     82.35        79   
166      166    Chaykovskiy      RU  56.77   54.11     66.99        92   

     Cloudiness  Wind Speed Current Description  Rain (inches)  Snow (inches)  \
13          100        9.40       moderate rain           2.91              0   
25          100        1.79          light rain           0.23              0   
42           46       15.99          light rain           0.25              0   
61          100        2.64          light rain           0.48              0   
107         100       11.39       moderate rain           2.43              0   
109          70        5.30          light rain           0.33              0   
110          96       18.97          light rain           0.12              0   
126         100       10.02          light rain           0.56              0   
158          84       12.10          light rain           0.11              0   
166          98        1.99          light rain           0.13              0   

    Hotel Name  
13              
25              
42              
61              
107             
109             
110             
126             
158             
166

In [23]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


In [24]:
# Iterate through the DataFrame.
for index, row in cities_hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        cities_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
cities_hotel_df.head(10)

City_ID           City Country    Lat     Lng  Max Temp  Humidity  \
13        13         Albany      US  42.60  -73.97     82.99        65   
25        25        Konongo      GH   6.62   -1.22     71.42        95   
42        42        Merauke      ID  -8.47  140.33     77.72        87   
61        61  Lengshuijiang      CN  27.69  111.43     76.41        93   
107      107     Butaritari      KI   3.07  172.79     83.10        75   
109      109         Padang      ID  -0.95  100.35     77.54        81   
110      110         Atuona      PF  -9.80 -139.03     79.57        79   
126      126         Kozhva      RU  65.11   57.05     67.77        91   
158      158       Lorengau      PG  -2.02  147.27     82.35        79   
166      166    Chaykovskiy      RU  56.77   54.11     66.99        92   

     Cloudiness  Wind Speed Current Description  Rain (inches)  Snow (inches)  \
13          100        9.40       moderate rain           2.91              0   
25          100        1.79          light rain           0.23              0   
42           46       15.99          light rain           0.25              0   
61          100        2.64          light rain           0.48              0   
107         100       11.39       moderate rain           2.43              0   
109          70        5.30          light rain           0.33              0   
110          96       18.97          light rain           0.12              0   
126         100       10.02          light rain           0.56              0   
158          84       12.10          light rain           0.11              0   
166          98        1.99          light rain           0.13              0   

                                  Hotel Name  
13                                            
25                           Alice Tee Hotel  
42                                            
61                                  Sheraton  
107                       Isles Sunset Lodge  
109                           Mercure Padang  
110                              Villa Enata  
126  Gostinicha IP Kayuda Olga Aleksandrovna  
158               Lorengau Harbourside Hotel  
166      Гостеприимные квартиры на Ленина 40

In [33]:
# add a pop-up marker for each city that displays the hotel name, city name, country,
# current weather description and maximum temperature.
# Define template for pop-up box
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row. use the iterrows() function to get the index and data in the row to add to the marker.
# hotel_info is a list
hotel_info = [info_box_template.format(**row) for index, row in cities_hotel_df.iterrows()]


In [34]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
# Get the latitude and longitude.
locations = cities_hotel_df[["Lat", "Lng"]]
max_temp = cities_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))